In [1]:
# %pip install tflite-model-maker
# %pip install tflite-runtime

In [2]:
# https://www.tensorflow.org/lite/tutorials/model_maker_image_classification
from __future__ import absolute_import, division
from BashColors import C
from CV2_Utils_2 import *
from TarfileFunctions import *

from time import perf_counter, sleep
import itertools, os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '5'

import numpy as np
import tensorflow as tf
assert tf.__version__.startswith('2')

from tflite_model_maker import model_spec
from tflite_model_maker import image_classifier
from tflite_model_maker.config import ExportFormat
from tflite_model_maker.config import QuantizationConfig
from tflite_model_maker.image_classifier import DataLoader

# from tflite_support.metadata_writers import image_classifier
# from tflite_support.metadata_writers import writer_utils
import matplotlib.pyplot as plt

contentPath=os.getcwd()
image_path=os.path.join(contentPath, 'images')
generatorPath=os.path.join(contentPath, 'DataGenerator')
imagePath=join(contentPath, '3b7d7d8a64.jpg')
generatorPath

'/home/jovyan/DataGenerator'

In [3]:
data = DataLoader.from_folder(generatorPath)
train_ds, restData = data.split(0.8)
val_ds, test_ds = restData.split(0.5)
print()
print('\nnum_classes:', train_ds.num_classes)
print('class names:', train_ds.index_to_label)

print(len(train_ds) // 8, len(val_ds)//8)

INFO:tensorflow:Load image with size: 844, num_label: 3, labels: Enemy, Friendly, Planets.


num_classes: 3
class names: ['Enemy', 'Friendly', 'Planets']
84 10


In [4]:
BATCH_SIZE = 16
EPOCHS = 1
DROPOUT_RATE = 0.5
LEARNING_RATE = 0.01
MOMENTUM = 0.9
start=perf_counter()
model = image_classifier.create(
    train_data = train_ds,
    validation_data = val_ds,
    model_spec = model_spec.get('mobilenet_v2'),
    learning_rate = LEARNING_RATE,
    batch_size = BATCH_SIZE,
    epochs = EPOCHS,
    dropout_rate = DROPOUT_RATE,
    use_augmentation = True, # default=False
    use_hub_library = True,  # default=True
    momentum = MOMENTUM, # Only used when use_hub_library is True
    shuffle = False,
    train_whole_model = False)

finish=perf_counter()
cvu.printTime(start, finish)

INFO:tensorflow:Retraining the models...
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 hub_keras_layer_v1v2 (HubKe  (None, 1280)             2257984   
 rasLayerV1V2)                                                   
                                                                 
 dropout (Dropout)           (None, 1280)              0         
                                                                 
 dense (Dense)               (None, 3)                 3843      
                                                                 
Total params: 2,261,827
Trainable params: 3,843
Non-trainable params: 2,257,984
_________________________________________________________________
None
INFO:tensorflow:Use default resize_bicubic.


INFO:tensorflow:Use default resize_bicubic.


INFO:tensorflow:Use customized resize method bilinear


INFO:tensorflow:Use customized resize method bilinear


INFO:tensorflow:Use customized resize method bilinear


INFO:tensorflow:Use customized resize method bilinear
/srv/conda/envs/notebook/lib/python3.7/site-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


42/42 [==============================] - 59s 1s/step - loss: 0.7088 - accuracy: 0.8735 - val_loss: 0.4777 - val_accuracy: 1.0000
completed: 1.0 minutes 31.0 second(s)


In [5]:
loss, accuracy = model.evaluate(test_ds)
# accuracy = round(accuracy, 1)
print(f'loss: {loss}\taccuracy: {accuracy*100}%')

INFO:tensorflow:Use customized resize method bilinear


INFO:tensorflow:Use customized resize method bilinear


3/3 [==============================] - 7s 2s/step - loss: 0.4853 - accuracy: 0.9882
loss: 0.4852568805217743	accuracy: 98.82352948188782%


In [6]:
fileName = 'Defcon4_mobilenet_v2_ObjectClassifier.tflite'
config = QuantizationConfig.for_float16()

model.export(export_dir=contentPath,
             tflite_filename=fileName,
             quantization_config=config)

INFO:tensorflow:Assets written to: /tmp/tmp9qzihpod/assets


INFO:tensorflow:Assets written to: /tmp/tmp9qzihpod/assets


INFO:tensorflow:Label file is inside the TFLite model with metadata.


INFO:tensorflow:Label file is inside the TFLite model with metadata.


INFO:tensorflow:Saving labels in /tmp/tmpmi3n4gcr/labels.txt


INFO:tensorflow:Saving labels in /tmp/tmpmi3n4gcr/labels.txt


INFO:tensorflow:TensorFlow Lite model exported successfully: /home/jovyan/Defcon4_mobilenet_v2_ObjectClassifier.tflite


INFO:tensorflow:TensorFlow Lite model exported successfully: /home/jovyan/Defcon4_mobilenet_v2_ObjectClassifier.tflite


In [8]:
help(model.export)

Help on method export in module tensorflow_examples.lite.model_maker.core.task.custom_model:

export(export_dir, tflite_filename='model.tflite', label_filename='labels.txt', vocab_filename='vocab.txt', saved_model_filename='saved_model', tfjs_folder_name='tfjs', export_format=None, **kwargs) method of tensorflow_examples.lite.model_maker.core.task.image_classifier.ImageClassifier instance
    Converts the retrained model based on `export_format`.
    
    Args:
      export_dir: The directory to save exported files.
      tflite_filename: File name to save tflite model. The full export path is
        {export_dir}/{tflite_filename}.
      label_filename: File name to save labels. The full export path is
        {export_dir}/{label_filename}.
      vocab_filename: File name to save vocabulary. The full export path is
        {export_dir}/{vocab_filename}.
      saved_model_filename: Path to SavedModel or H5 file to save the model. The
        full export path is
        {export_dir}/{sa

In [9]:
model.export(
    export_dir=contentPath, export_format=ExportFormat.LABEL)

INFO:tensorflow:Saving labels in /home/jovyan/labels.txt


INFO:tensorflow:Saving labels in /home/jovyan/labels.txt


In [10]:
class TensorflowLiteClassificationModel:
    def __init__(self, model_path, labels, image_size=224):
        self.interpreter = tf.lite.Interpreter(model_path=model_path)
        self.interpreter.allocate_tensors()
        self._input_details = self.interpreter.get_input_details()
        self._output_details = self.interpreter.get_output_details()
        self.labels = labels
        self.image_size=image_size

    def run_from_filepath(self, image_path):
        input_data_type = self._input_details[0]["dtype"]
        image = np.array(Image.open(image_path).resize((self.image_size, self.image_size)), dtype=input_data_type)
        if input_data_type == np.float32:
            image = image / 255.

        if image.shape == (1, 224, 224):
            image = np.stack(image*3, axis=0)

        return self.run(image)

    def run(self, image):
        """
        args:
          image: a (1, image_size, image_size, 3) np.array

        Returns list of [Label, Probability], of type List<str, float>
        """

        self.interpreter.set_tensor(self._input_details[0]["index"], image)
        self.interpreter.invoke()
        tflite_interpreter_output = self.interpreter.get_tensor(self._output_details[0]["index"])
        probabilities = np.array(tflite_interpreter_output[0])

        # create list of ["label", probability], ordered descending probability
        label_to_probabilities = []
        for i, probability in enumerate(probabilities):
            label_to_probabilities.append([self.labels[i], float(probability)])
        return sorted(label_to_probabilities, key=lambda element: element[1])

In [ ]:
# Usage
modelPath='/home/jovyan/Defcon4_mobilenet_v2_ObjectClassifier.tflite'
imagePath=join(contentPath, '3b7d7d8a64.jpg')
labelsPath=join(contentPath, 'labels.txt')
model = TensorflowLiteClassificationModel(
    model_path=modelPath, labels=labelsPath)
img=cvu.getCV2Image(imagePath)
(label, probability) = model.run()

In [ ]:
# modelPath='/home/jovyan/Defcon4_mobilenet_v2_ObjectClassifier.tflite'
import tensorflow as tf
import tflite_runtime.interpreter as tflite
interpreter = tf.lite.Interpreter(model_path=modelPath)

interpreter

In [ ]:
import numpy as np
import tensorflow as tf

modelPath='/home/jovyan/Defcon4_mobilenet_v2_ObjectClassifier.tflite'
# Load the TFLite model and allocate tensors.
interpreter = tf.lite.Interpreter(model_path=modelPath)
interpreter.allocate_tensors()

# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Test the model on random input data.
input_shape = input_details[0]['shape']
input_data = np.array(np.random.random_sample(input_shape), dtype=np.float32)
interpreter.set_tensor(input_details[0]['index'], input_data)

interpreter.invoke()

# The function `get_tensor()` returns a copy of the tensor data.
# Use `tensor()` in order to get a pointer to the tensor.
output_data = interpreter.get_tensor(output_details[0]['index'])
print(output_data)